In [1]:
import sys
sys.path.append('/mnt/d/ariel2/code/core/')
sys.path.append('d:/ariel2/code/core/')
sys.path.append('/kaggle/input/my-ariel2-library')
import kaggle_support as kgs
import ariel_model
import ariel_gp
import time
import numpy as np
import cupy as cp
import multiprocess
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
import ariel_numerics
import ariel_load

local


In [2]:
train_data = kgs.load_all_train_data()
print(len(train_data))

1099


In [ ]:
# FGS = []
# AIRS = []
# N_FGS=2500
# N_AIRS=250
# loaders = ariel_load.default_loaders()
# for d in tqdm(train_data):
#     for t in d.transits:
#         tt = copy.deepcopy(t)
#         tt.load_to_step(2, d, loaders)
#         for ii in range(tt.data[0].data.shape[0]//N_FGS):
#             FGS.append(cp.mean(tt.data[0].data[N_FGS*ii:(N_FGS)*(ii+1),...],0).get())
#         for ii in range(tt.data[1].data.shape[0]//N_AIRS):
#             AIRS.append(cp.mean(tt.data[1].data[N_AIRS*ii:(N_AIRS)*(ii+1),...],0).get())
# del tt
# FGS = cp.array(FGS)
# AIRS = cp.array(AIRS)
# data = [FGS, AIRS]
# kgs.dill_save(kgs.temp_dir + '/explore_bad_pixels.pickle', data)
data_modes = kgs.dill_load(kgs.temp_dir + '/explore_bad_pixels.pickle')

  2%|█▌                                                                               | 22/1099 [00:33<31:11,  1.74s/it]

In [ ]:
n_components = [10]+[5]*282
S = []
components = []
for i_wavelength in range(3):
    if i_wavelength==0:
        this_data = copy.deepcopy(data_modes[0].reshape(-1,1024))
    else:
        this_data = copy.deepcopy(data_modes[1][:,:,i_wavelength-1])
    print(i_wavelength, end =" ")
    dat = this_data
    S.append([]);components.append([]);
    for ii in range(n_components[i_wavelength]):
        this_weights, this_components, this_S = ariel_numerics.nan_pca(dat, 1, tol=1e-1, verbose=False)
        #print(this_S)
        #dat = dat - weights[:,0][:,None,None]*components[0,:][None,:,:]
        dat -= this_weights * this_components
        S[-1].append(this_S)
        components[-1].append(this_components)
        #plt.figure();plt.semilogy(S.get())
        #weights,components,S = ariel_numerics.nan_pca(data_residual, 4, tol=1e-1, verbose=True)
        #plt.figure();plt.semilogy(S.get())
        #weights,components,S = ariel_numerics.nan_pca(data_residual, 5, tol=1e-1, verbose=True)
    S[-1]=cp.array(S[-1])[:,0]

In [ ]:
plt.figure()
plt.semilogy(S[0].get())
plt.figure()
SS = cp.stack(S[1:]).T
plt.imshow(cp.log(SS).get()/np.log(10), aspect='auto')
plt.colorbar()

In [ ]:
#plt.plot(cp.nanmean(all_b,1).get())
print(np.max(cp.nanmedian(all_b,0).get()))
ind = np.argmax(cp.nanmedian(all_b,0).get())
plt.plot(all_b[:,ind].get())
np.nanmedian(all_b[:,ind].get())
#plot_all(all_a[ind,:])
#plot_all(all_b[ind,:])

In [ ]:
import ariel_numerics
import importlib
importlib.reload(ariel_numerics)
weights,components,S = ariel_numerics.nan_pca(all_a-a, 10, tol=1e-5, verbose=True)
plt.figure();plt.plot(S.get())
plt.figure();plt.imshow(a.reshape(32,32).get())
plt.figure();plt.imshow(components[0,:].reshape(32,32).get())
plt.figure();plt.imshow(components[1,:].reshape(32,32).get())

In [ ]:
import ariel_numerics
import importlib
importlib.reload(ariel_numerics)
weights2,components2,S2 = ariel_numerics.nan_pca(all_b-b, 10, tol=1e-5, verbose=True)
plt.figure();plt.plot(S.get())
plt.figure();plt.imshow(b.reshape(32,32).get())
plt.figure();plt.imshow(components2[0,:].reshape(32,32).get())
plt.figure();plt.imshow((components2[1,:]).reshape(32,32).get())

In [ ]:
weights.shape

In [ ]:
S,S2